In [1]:
import torch
import os
import matplotlib.pyplot as plt
os.chdir("..")

from epilearn.models.SpatialTemporal.STGCN import STGCN

from epilearn.models.Spatial.GCN import GCN
from epilearn.models.Spatial.SAGE import SAGE
from epilearn.models.Spatial.GAT import GAT
from epilearn.models.Spatial.DCRNN import DCRNN
from epilearn.models.Spatial.GIN import GIN

from epilearn.data import UniversalDataset
from epilearn.utils import utils, transforms
from epilearn.utils import simulation
from epilearn.tasks.detection import Detection

### Configs

In [2]:
# initial settings
device = torch.device('cpu')
torch.manual_seed(7)

lookback = 1 # inputs size
horizon = 2 # predicts size; also seen as number of classes

epochs = 50 # training epochs
batch_size = 25 # training batch size

### Initialize Dataset

In [3]:
# load toy dataset
dataset = UniversalDataset()
dataset.load_toy_dataset()

### Initialize model and task

In [4]:
task = Detection(prototype=GCN, dataset=None, lookback=lookback, horizon=horizon, device='cpu')

### Add transformations

In [5]:
# transformation = transforms.Compose({
#                                  'features':[transforms.normalize_feat()], 
#                                  'graph': [transforms.normalize_adj()], 
#                                  'dynamic_graph': [transforms.normalize_adj()], 
#                                  'states': []
#                                  })
transformation = transforms.Compose({
                                 'features':[], 
                                 'graph': [], 
                                 'dynamic_graph': [], 
                                 'states': []
                                 })
dataset.transforms = transformation

### Train model

In [6]:
config = None
result = task.train_model(dataset=dataset, config=config, loss='ce', epochs=5) # instead of config, we can also dircetly input some parameters

spatial model loaded!


 20%|██        | 1/5 [00:00<00:00,  8.33it/s]

######### epoch:0
Training loss: 1.940663456916809
Validation loss: 3.3824844360351562
######### epoch:1
Training loss: 1.2578233480453491
Validation loss: 1.3962855339050293


 60%|██████    | 3/5 [00:00<00:00,  9.31it/s]

######### epoch:2
Training loss: 0.8243905901908875
Validation loss: 1.1286555528640747


...

######### epoch:3
Training loss: 0.582511842250824
Validation loss: 0.7725021839141846


100%|██████████| 5/5 [00:00<00:00,  9.45it/s]


######### epoch:4
Training loss: 0.49703678488731384
Validation loss: 0.7213609218597412

Final Training loss: 0.49703678488731384
Final Validation loss: 0.7213609218597412
Best Epoch: 4
Best Training loss: 0.49703678488731384
Best Validation loss: 0.7213609218597412

Predicting Progress...


100%|██████████| 108/108 [00:00<00:00, 1229.30it/s]

Test ACC: 2.194444417953491


### Train on Simulated dataset

In [7]:
# Simulation Process
from epilearn.models.SpatialTemporal.NetworkSIR import NetSIR

# generate 10 samples
num_nodes = 25
# generate random static graph: 25 nodes
initial_graph = simulation.get_random_graph(num_nodes=num_nodes, connect_prob=0.15)
initial_states = torch.zeros(num_nodes,3) # [S,I,R]
initial_states[:, 0] = 1

graph = initial_graph
x = []
y = []
for i in range(100): 
    # set infected individual
    idx = torch.randint(0,num_nodes, (1,))
    initial_states[idx.item(), 0] = 0
    initial_states[idx.item(), 1] = 1

    model = NetSIR(num_nodes=initial_graph.shape[0], horizon=100, infection_rate=0.01, recovery_rate=0.0384) # infection_rate, recover_rate, fixed_population
    preds = model(initial_states, initial_graph, steps = None)
    x.append(torch.nn.functional.one_hot(preds[-1].argmax(1)))
    y.append(initial_states.argmax(1))
x = torch.stack(x)
y = torch.stack(y)

In [8]:
dataset = UniversalDataset(x=x,y=y,graph=initial_graph)
dataset.transforms = transformation
task = Detection(prototype=GCN, dataset=dataset, lookback=lookback, horizon=horizon, device='cpu')

In [9]:
result = task.train_model(dataset=dataset, loss='ce', epochs=5)

spatial model loaded!


100%|██████████| 5/5 [00:00<00:00, 102.85it/s]


######### epoch:0
Training loss: 0.6525769233703613
Validation loss: 0.5134344100952148
######### epoch:1
Training loss: 0.6488687992095947
Validation loss: 0.5087106227874756
######### epoch:2
Training loss: 0.6446580290794373
Validation loss: 0.5027973651885986
######### epoch:3
Training loss: 0.6400063037872314
Validation loss: 0.4960256814956665
######### epoch:4
Training loss: 0.6372864246368408
Validation loss: 0.4883958101272583

Final Training loss: 0.6372864246368408
Final Validation loss: 0.4883958101272583
Best Epoch: 4
Best Training loss: 0.6372864246368408
Best Validation loss: 0.4883958101272583

Predicting Progress...


100%|██████████| 20/20 [00:00<00:00, 2434.59it/s]

Test ACC: 23.0
